# 🔍 KAS_Verifier_DualMode.ipynb
Verify a `.sha256` file against a `.sig` signature using:
- ✅ OpenSSL (command line)
- 🐍 Python with `cryptography` (secure fallback)

You will need to upload:
- `.sha256` file (plain hash only)
- `.sig` signature file
- `public_key.pem`

In [ ]:
# 📤 Upload required files
from google.colab import files
uploaded = files.upload()

sha256_file = [f for f in uploaded if f.endswith('.sha256')][0]
sig_file = [f for f in uploaded if f.endswith('.sig')][0]
key_file = [f for f in uploaded if f.endswith('.pem')][0]

In [ ]:
# 📌 Choose method: OpenSSL or Python
method = input("Use OpenSSL (o) or Python (p)? [o/p]: ").strip().lower()
assert method in ['o', 'p'], "Invalid choice. Use 'o' or 'p'."

In [ ]:
# ✅ OpenSSL Mode
if method == 'o':
    print("🔍 Verifying using OpenSSL...")
    !openssl dgst -sha256 -verify {key_file} -signature {sig_file} {sha256_file}

In [ ]:
# 🐍 Python Verification Mode
if method == 'p':
    !pip install --quiet cryptography
    from cryptography.hazmat.primitives import serialization, hashes
    from cryptography.hazmat.primitives.asymmetric import padding
    from cryptography.exceptions import InvalidSignature
    from cryptography.hazmat.backends import default_backend

    with open(key_file, "rb") as f:
        public_key = serialization.load_pem_public_key(f.read(), backend=default_backend())

    with open(sha256_file, "rb") as f:
        hash_bytes = f.read().strip()

    with open(sig_file, "rb") as f:
        signature = f.read()

    try:
        public_key.verify(
            signature,
            hash_bytes,
            padding.PKCS1v15(),
            hashes.SHA256()
        )
        print("✅ Signature is valid.")
    except InvalidSignature:
        print("❌ Signature verification failed.")